# **Data preparation**

## **Generating non redundant datasets**

In [230]:
import pandas as pd

In [284]:
def filter_redundancy(unique_ids, total_dataset):

    unique_ids = pd.read_csv(unique_ids, header=None, sep='\t')
    total_df = pd.read_csv(total_dataset, header=None, sep='\t')
    filtered = total_df[total_df.iloc[:,0].isin(unique_ids.iloc[:,0])]
    
    #add write file 
    
    return filtered

non_redundant_pos = filter_redundancy('../data_preparation/non_redundant/uniq.pos.tsv', '../data_collection/positive.tsv')

non_redundant_neg = filter_redundancy('../data_preparation/non_redundant/uniq.neg.tsv', '../data_collection/negative.tsv')

## **Dividing in train and test set**

In [275]:
def split_train_test(df: pd.DataFrame, split: int):
    random_seed = 42
    shuffle_df = df.sample(frac=1, random_state=random_seed)
    split_idx = split*len(df.index)//100
    training_set = df.iloc[:split_idx,:].copy()
    test_set = df.iloc[split_idx:,:].copy()
    return training_set, test_set
    
training_set_pos, test_set_pos = split_train_test(non_redundant_pos, 80)
training_set_neg, test_set_neg = split_train_test(non_redundant_neg, 80)

In [285]:
print(f'the leanght of the positive set is {len(training_set_pos)+len(test_set_pos)}; training:{len(training_set_pos)}, testing{len(test_set_pos)}')
print(f'the leanght of the negative set is {len(training_set_neg)+len(test_set_neg)}; training:{len(training_set_neg)}, testing{len(test_set_neg)}')


the leanght of the positive set is 1093; training:874, testing219
the leanght of the negative set is 8934; training:7147, testing1787


In [277]:
bins = [0, 25, 50, 75, 100]
bin_lables = [1, 2, 3, 4, 5]
training_set_pos.loc[:, 'temp_row_num'] = training_set_pos.index.to_numpy(dtype=np.int64)

training_set_pos.loc[:, 'validation_n'] = pd.qcut(training_set_pos['temp_row_num'], q=5 , labels=bin_lables )


training_set_neg.loc[:, 'temp_row_num'] = training_set_neg.index.to_numpy(dtype=np.int64)

training_set_neg.loc[:, 'validation_n'] = pd.qcut(training_set_neg['temp_row_num'], q=5 , labels=bin_lables )

In [278]:
training_set_neg['validation_n'].nunique

<bound method IndexOpsMixin.nunique of 0        1
4        1
7        1
11       1
12       1
        ..
17323    5
17324    5
17325    5
17327    5
17331    5
Name: validation_n, Length: 7147, dtype: category
Categories (5, int64): [1 < 2 < 3 < 4 < 5]>